In [30]:
import os
from dotenv import load_dotenv
load_dotenv()

import warnings
warnings.filterwarnings("ignore")

# Langsmith Tracing
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['LANGCHAIN_PROJECT'] = os.getenv('LANGCHAIN_PROJECT')
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')

In [4]:
from langchain_groq import ChatGroq
from pprint import pprint

llm = ChatGroq(model='groq/compound-mini')
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001997D49E8D0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001997D52A870>, model_name='groq/compound-mini', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [5]:
from  langchain_core.output_parsers import StrOutputParser 
res = llm.invoke('Can you explain me about rules of f1 in 10 lines')
parser = StrOutputParser()
for line in parser.invoke(res).split("\n"):
    print(line)

Here are the rules of F1 in 10 lines:

1. Formula 1 (F1) is a single-seater racing championship with 20 cars and 10 teams.
2. Each team has 2 drivers competing in Grands Prix around the world.
3. Cars are designed and built by teams, with engines supplied by 4 manufacturers.
4. Qualifying sessions determine the starting grid for each Grand Prix.
5. The objective is to complete a set number of laps in the shortest time possible.
6. Drivers can make pit stops to change tires and refuel (not always necessary).
7. Overtaking is allowed, but drivers must stay on track and avoid contact.
8. Drivers are penalized for infractions like speeding in the pit lane or causing collisions.
9. Points are awarded to the top 10 finishers: 25, 18, 15, 12, 10, 8, 6, 4, 2, 1.
10. The driver and team with the most points at the end of the season are crowned World Champions.


### HumanMessages & System Messages

In [6]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
                SystemMessage(content="Translate the user message to French"),
                HumanMessage(content="You look beautiful tonight!")
]

response = llm.invoke(messages)
response.content

"You look beautiful tonight translates to:\n\nTu as l'air magnifique ce soir!\n\nor, in a more formal tone:\n\nVous êtes magnifique ce soir!"

In [7]:
from langchain_core.output_parsers import StrOutputParser

str_parser = StrOutputParser()
str_parser.invoke(response)

"You look beautiful tonight translates to:\n\nTu as l'air magnifique ce soir!\n\nor, in a more formal tone:\n\nVous êtes magnifique ce soir!"

In [8]:
# Using LCEL For Chaining

chain1 = llm|str_parser
chain1.invoke(messages)

'Tu es belle ce soir!'

In [9]:
from langchain_core.prompts import ChatPromptTemplate

gen_prompt  = "Translate the following into {language}"
prompt = ChatPromptTemplate.from_messages([
                ('system',gen_prompt),
                ("user", "{text}")
])

prompt.invoke({"language":"French","text":"Rahul is a really amazing person"})

ChatPromptValue(messages=[SystemMessage(content='Translate the following into French', additional_kwargs={}, response_metadata={}), HumanMessage(content='Rahul is a really amazing person', additional_kwargs={}, response_metadata={})])

In [10]:
chain2  = prompt|llm|str_parser
chain2.invoke({"language":"French","text":"Rahul is a really amazing person"})

'Rahul est une personne vraiment incroyable.'

In [11]:
# Simple Demonstration of how the History is stored 
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

llm.invoke([
    HumanMessage("Hi Im Rahul, Im a great AI Engineer"),
    AIMessage("Hello Rahul! Nice to meet you! It's great to hear that you're an AI engineer. What kind of AI projects have you been working on lately"),
    HumanMessage("Hey whats my name and what do I Do?")
])

AIMessage(content="Your name is Rahul, and you mentioned you're a great AI Engineer!", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 439, 'total_tokens': 471, 'completion_time': 0.058586, 'prompt_time': 0.069088, 'queue_time': 0.100453, 'total_time': 0.127674}, 'model_name': 'groq/compound-mini', 'system_fingerprint': None, 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--7f891b6b-d3b1-4197-a997-f54e69723a6f-0', usage_metadata={'input_tokens': 439, 'output_tokens': 32, 'total_tokens': 471})

### Chat Message History

In [ ]:
# Base code for chat history
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import StrOutputParser

store = {}

def get_session_history(session_id :str)->BaseChatMessageHistory: # This is the base message history
    if session_id not in store:
        store[session_id] = ChatMessageHistory() # This is the active session history
    return store[session_id]

with_chat_message_history = RunnableWithMessageHistory(llm,get_session_history)

config = {"configurable":{"session_id":"chat1"}}

response = with_chat_message_history.invoke([HumanMessage(content="Hi My Name is Rahul")],
                                 config=config)

op_parser = StrOutputParser()
op_parser.invoke(response)

"Hello Rahul! It's nice to meet you. Is there something I can help you with or would you like to chat?"

### Using  ChatPromptTemplate

    System Message - Prompt Given by the user
    Human Message - Input Given by the user
    AI Message - Output generated by llm 

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_prompt = ChatPromptTemplate.from_messages([
    ("system","Answer the question in simple understandable terms"),
    MessagesPlaceholder(variable_name="text")
])

chain = chat_prompt|llm|op_parser

chain.invoke([HumanMessage(content="Hi My Name is Rahul")])

"Hello Rahul! It's nice to meet you. Is there something I can help you with or would you like to chat?"

In [17]:
with_chat_message_history = RunnableWithMessageHistory(chain, get_session_history)

with_chat_message_history.invoke([HumanMessage("What is the meaning of my name")],
                                 config=config)

'Rahul is a popular name in India and other parts of South Asia. The name Rahul has multiple meanings depending on the language and culture. Here are a few possible interpretations:\n\n* In Sanskrit, "Rahul" means "efficient" or "effective".\n* In Buddhism, Rahul is considered a name that means "conqueror of all faults".\n\nOverall, the name Rahul is often associated with positive qualities like strength, efficiency, and wisdom. What do you think about your name, Rahul?'

In [18]:
with_chat_message_history.invoke([HumanMessage("I also found that meaning of my name is Son of Gods")],
                                 config=config)

'That\'s a great addition! In Hindu mythology, Rahul is indeed also considered to mean "Son of God" or "Efficacious" or even "Traveler". \n\nIn Buddhist tradition, Rahul is the name of the son of Lord Buddha, which further solidifies the connection to the divine.\n\nSo, it seems your name, Rahul, carries a rich history and profound meaning. How do you feel about being a "Son of God"?'

### Using Multiple inputs in Prompt

In [24]:
prompt2 = ChatPromptTemplate.from_messages([
    ("system","Perform the following {task} on the information"),
    MessagesPlaceholder(variable_name="messages")
])

config2 = {"configurable":{"session_id":"chat2"}}

chain2 = prompt2|llm|op_parser

chat_with_history_2 = RunnableWithMessageHistory(chain2, get_session_history, config=config2, input_messages_key="messages")

res = chat_with_history_2.invoke({"messages":[HumanMessage("Hi My name is Rahul. Im learning AI. What skills do I need to excel in my field")],
                            "task":"generate"}, config=config2)
for i in res.split("\n"):
    print(i)

Nice to meet you, Rahul!

To excel in AI, focus on developing these key skills:

**Top 5 Technical Skills:**

1. **Python Programming**: Master Python and its popular libraries like NumPy, pandas, and scikit-learn.
2. **Machine Learning**: Study supervised and unsupervised learning, neural networks, and deep learning.
3. **Mathematics**: Linear Algebra, Calculus, Probability, and Statistics are essential math concepts for AI.
4. **Deep Learning Frameworks**: Familiarize yourself with TensorFlow, PyTorch, or Keras.
5. **Data Preprocessing**: Understand data cleaning, feature engineering, and data visualization.

**Top 3 Soft Skills:**

1. **Communication**: Develop clear and concise communication skills to explain complex AI concepts.
2. **Problem-Solving**: Improve your critical thinking and problem-solving skills.
3. **Collaboration**: Learn to work effectively with cross-functional teams.

**Additional Tips:**

* Stay updated with the latest AI research and trends.
* Practice with Ka

In [25]:
chat_with_history_2.invoke({"messages":[HumanMessage("What skills do I need to excel in my field")],
                            "task":"summarize"}, config=config2)

"To excel in AI, focus on developing a combination of technical and soft skills. Here are the key ones:\n\n**Technical Skills:**\n\n1. **Programming**: Python, Java, or C++.\n2. **Machine Learning**: Supervised and unsupervised learning, neural networks, deep learning.\n3. **Mathematics**: Linear Algebra, Calculus, Probability, Statistics.\n4. **Data Preprocessing**: Data cleaning, feature engineering, data visualization.\n5. **Deep Learning Frameworks**: TensorFlow, PyTorch, Keras.\n\n**Soft Skills:**\n\n1. **Communication**: Clear and concise communication.\n2. **Problem-Solving**: Critical thinking and problem-solving.\n3. **Collaboration**: Effective teamwork.\n\n**Additional Skills (nice to have):**\n\n1. **Cloud Computing**: AWS, Google Cloud, Azure.\n2. **Big Data**: Hadoop, Spark, NoSQL databases.\n3. **Computer Vision**: Image processing, object detection.\n4. **Natural Language Processing (NLP)**: Text processing, sentiment analysis.\n\n**Stay Up-to-Date:**\n\n1. **Follow AI 

In [ ]:
# Sample Groq API Call
llama4 = ChatGroq(model='meta-llama/llama-4-maverick-17b-128e-instruct')
res = llama4.invoke("What is newton raphson method") 
res = op_parser.invoke(res)
res

'**Newton-Raphson Method**\n=========================\n\nThe Newton-Raphson method is a powerful iterative technique used to find the roots of a real-valued function. It is a popular method for solving nonlinear equations and is widely used in various fields, including mathematics, physics, engineering, and computer science.\n\n**How it Works**\n----------------\n\nThe Newton-Raphson method starts with an initial guess, `x0`, for the root of the function `f(x)`. It then iteratively improves this estimate using the formula:\n\n`x_n+1 = x_n - f(x_n) / f\'(x_n)`\n\nwhere `f\'(x)` is the derivative of `f(x)`.\n\n**Step-by-Step Process**\n-------------------------\n\n1. **Choose an initial guess**: Select an initial value `x0` for the root.\n2. **Evaluate the function and its derivative**: Compute `f(x0)` and `f\'(x0)`.\n3. **Update the estimate**: Use the Newton-Raphson formula to compute `x1 = x0 - f(x0) / f\'(x0)`.\n4. **Repeat the process**: Continue iterating until convergence, i.e., u

In [ ]:
from langchain_community.embeddings import OllamaEmbeddings
ollama_large_embedding_model = OllamaEmbeddings(model="mxbai-embed-large")

# For loading the embedding model, Need to run this command before executing
# !ollama pull mxbai-embed-large

ollama_large_embedding_model.embed_query("Heya dude")

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest 
pulling 819c2adf5ce6: 100% ▕██████████████████▏ 669 MB                         
pulling c71d239df917: 100% ▕██████████████████▏  11 KB                         
pulling b837481ff855: 100% ▕██████████████████▏   16 B                         
pulling 38badd946f91: 100% ▕██████████████████▏  408 B                         
verifying sha256 digest 
writing manifest 
success 


[1.1137117147445679,
 0.17145174741744995,
 -0.17546609044075012,
 0.6683796048164368,
 -0.04627767205238342,
 -0.2045161873102188,
 0.9842468500137329,
 0.7631851434707642,
 0.5983991026878357,
 0.7992069721221924,
 -0.04209443926811218,
 -0.1315927803516388,
 -0.8873213529586792,
 0.2068871706724167,
 -1.0761505365371704,
 -0.3576597273349762,
 -0.20002833008766174,
 -0.4431720972061157,
 -0.2306552678346634,
 0.4796924591064453,
 0.17272156476974487,
 0.8530593514442444,
 -1.103057622909546,
 -0.026106566190719604,
 -0.03488060086965561,
 -0.020625941455364227,
 -0.735515296459198,
 0.19991517066955566,
 0.66607666015625,
 1.0564184188842773,
 -0.1497926115989685,
 0.6771297454833984,
 -0.19747677445411682,
 -0.8436295986175537,
 -0.1921650469303131,
 -0.7078328132629395,
 0.6134791374206543,
 -0.8902954459190369,
 -0.5287086963653564,
 -0.24808502197265625,
 0.15274302661418915,
 0.44245290756225586,
 0.5816109776496887,
 -0.4918016195297241,
 -1.3627941608428955,
 0.72602057456970

In [ ]:
# !ollama pull all-minilm:22m
mini_embedding = OllamaEmbeddings(model="all-minilm:22m")
mini_embedding.embed_query("Hey dude")

[-0.24060851335525513,
 0.22685018181800842,
 0.18480272591114044,
 0.23355406522750854,
 -0.0965418666601181,
 -0.4233628511428833,
 0.7404276132583618,
 0.24716417491436005,
 0.04158234968781471,
 0.035536471754312515,
 -0.10603231936693192,
 -0.16012725234031677,
 -0.1338970810174942,
 -0.32922735810279846,
 0.026884786784648895,
 0.06513835489749908,
 0.3278445899486542,
 -0.5141737461090088,
 -0.2242540866136551,
 0.18521517515182495,
 -0.1565600037574768,
 0.43680572509765625,
 -0.17750495672225952,
 -0.05638369545340538,
 -0.06916357576847076,
 0.1716945320367813,
 0.20684824883937836,
 0.21356680989265442,
 -0.03766027092933655,
 -0.1252012401819229,
 -0.3868488371372223,
 0.36807119846343994,
 0.06210847198963165,
 0.3698060214519501,
 0.1373519003391266,
 -0.1627090722322464,
 0.10827478021383286,
 -0.23959073424339294,
 -0.20146149396896362,
 -0.13275942206382751,
 0.22338126599788666,
 0.1603085696697235,
 0.0021697115153074265,
 0.26740968227386475,
 -0.01312724594026804,


In [44]:
# Loading Document using Wikipedia loader and then performing chunking in the same step
from langchain_community.document_loaders import WikipediaLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
chunked_docs = WikipediaLoader("Agentic AI",load_max_docs=1,  doc_content_chars_max=200).load_and_split(RecursiveCharacterTextSplitter(chunk_size = 100, chunk_overlap = 20))
chunked_docs

[Document(metadata={'title': 'Agentic AI', 'summary': 'Agentic AI is a class of artificial intelligence that focuses on autonomous systems that can make decisions and perform tasks with or without human intervention. The independent systems automatically respond to conditions, with procedural, algorithmic, and human-like creative steps, to produce process results. The field is closely linked to agentic automation, also known as agent-based process management systems, when applied to process automation. Applications include software development, customer support, cybersecurity and business intelligence. \n\n', 'source': 'https://en.wikipedia.org/wiki/Agentic_AI'}, page_content='Agentic AI is a class of artificial intelligence that focuses on autonomous systems that can make'),
 Document(metadata={'title': 'Agentic AI', 'summary': 'Agentic AI is a class of artificial intelligence that focuses on autonomous systems that can make decisions and perform tasks with or without human interventi

## Integrating Retrievers

In [ ]:
from langchain_chroma import Chroma
vector_store = Chroma.from_documents(chunked_docs, embedding=mini_embedding)
vector_store

    Info : Vectorstores are not runnable and hence cannot be integrated directly to chains
         Thatswhy we need to convert these vectorstores as retrievers which are runnable to be able to link to chains

In [46]:
retriever = vector_store.as_retriever(
    search_type= 'similarity',
    search_kwargs = {"k":1}
)
retriever.batch(['cybersecurity', 'automation'])

[[Document(id='bb8de4f9-4a5b-4498-82f0-d1f8a060f55e', metadata={'summary': 'Agentic AI is a class of artificial intelligence that focuses on autonomous systems that can make decisions and perform tasks with or without human intervention. The independent systems automatically respond to conditions, with procedural, algorithmic, and human-like creative steps, to produce process results. The field is closely linked to agentic automation, also known as agent-based process management systems, when applied to process automation. Applications include software development, customer support, cybersecurity and business intelligence. \n\n', 'title': 'Agentic AI', 'source': 'https://en.wikipedia.org/wiki/Agentic_AI'}, page_content='Agentic AI is a class of artificial intelligence that focuses on autonomous systems that can make')],
 [Document(id='bb8de4f9-4a5b-4498-82f0-d1f8a060f55e', metadata={'title': 'Agentic AI', 'summary': 'Agentic AI is a class of artificial intelligence that focuses on auto

In [49]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message  = """
Answer the question using provided context only

{question}

context :
{context}
"""

prompt = ChatPromptTemplate.from_messages(["human",message])
rag_chain = {'context':retriever, "question":RunnablePassthrough()} |prompt|llama4|op_parser
res = rag_chain.invoke("what is agentic ai")
res

'Agentic AI is a class of artificial intelligence that focuses on autonomous systems that can make decisions and perform tasks with or without human intervention.'